In [5]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [6]:
from shutil import copy2

In [7]:
import matplotlib.pyplot as plt

In [8]:
races = ['African','Asian','Caucasian','Indian']
labels = []
dist = []
for race in races:
        with open(f"./sims/{race}_sims.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [9]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
idx = np.argmax(tpr-fpr)
threshold = thresholds[idx]
print(f"Threshold: {threshold}")

Threshold: 0.30249398946762085


In [10]:
races = ['African','Asian','Caucasian','Indian']
accuracies = {}
for race in races:
        accuracies[f"{race}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./sims/{race}_sims.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}"]['TN'] += 1
                else:
                    accuracies[f"{race}"]['FN'] += 1
            else:
                with open(f"./results/{race}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}"]['TP'] += 1
                else:
                    accuracies[f"{race}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [11]:
print(accuracies)

{'African': {'TP': 2964, 'TN': 36, 'FP': 32, 'FN': 2968}, 'Asian': {'TP': 2914, 'TN': 86, 'FP': 22, 'FN': 2978}, 'Caucasian': {'TP': 2928, 'TN': 72, 'FP': 1, 'FN': 2999}, 'Indian': {'TP': 2931, 'TN': 69, 'FP': 27, 'FN': 2972}}


In [12]:
gar = {}
far = {}
scores = {}
for race in races:
        tp = accuracies[f"{race}"]['TP']
        tn = accuracies[f"{race}"]['TN']
        fp = accuracies[f"{race}"]['FP']
        fn = accuracies[f"{race}"]['FN']
        gar[f"{race}"] = tp/(tn+tp)
        far[f"{race}"] = fp/(fn+fp)
        scores[f"{race}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [13]:
gar

{'African': 0.988,
 'Asian': 0.9713333333333334,
 'Caucasian': 0.976,
 'Indian': 0.977}

In [14]:
far

{'African': 0.010666666666666666,
 'Asian': 0.007333333333333333,
 'Caucasian': 0.0003333333333333333,
 'Indian': 0.009003001000333445}

In [15]:
scores

{'African': 98.86666666666666,
 'Asian': 98.2,
 'Caucasian': 98.78333333333333,
 'Indian': 98.39973328888148}